<hr style="border:2px solid gray"> </hr>
<img src="https://mma.prnewswire.com/media/1095203/East_Tennessee_State_University_Logo.jpg?p=facebook" width=200 height=200 />

<div class="alert alert-block alert-info">
    <h1 style="text-align: center">CSCI 5037 - NLP & Text Analysis</h1>
</div>

# <center>Text Classification of Amazon Reviews</center>

<center>Mofeoluwa Jide-Jegede </center>
<center>Department of Computing </center>
<center>East Tennessee State University</center>

<hr style="border:2px solid lightblue"> </hr>

<div class="alert alert-block alert-info">

# Part 1: Activity 

</div>

In [43]:
#Imported the needed libraries for analysis
import warnings 
warnings.filterwarnings ('ignore')

import seaborn as sns
import re
import os 
import gzip 
from collections import Counter 
from pprint import pprint

import pandas as pd 
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt

np.random.seed(123)

import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from tensorflow import keras
from tensorflow.keras.layers import Dense
layer = Dense(10)
from tensorflow import keras

import string 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.text import TextCollection 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn 
from sklearn.manifold import TSNE 
from sklearn.metrics.pairwise import cosine_similarity


from tqdm.notebook import tqdm

import watermark

%load_ext watermark

%matplotlib inline

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


Read the content of the `AmazonReviews.csv` into a dataframe called `reviews_df` 

In [12]:
#Call the data from AmazonReviews.csv

reviews_df= pd.read_csv('AmazonReviews.csv')

In [13]:
#Inspect the colums in the dataframe
reviews_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
# Q1. 

1.1. Add a new column to the dataframe called `sentiment` calculated based on the `Score` field values as follows:
        1.1.1. if the Score value is less than 3 then the sentimnet is 0 `negative`
        1.1.2. if the Score value is greater than or equal to 3 then the sentimnet is 1 `positive`

In [14]:
#Create a column called 'Sentiment' from the 'Score' column

reviews_df.loc[reviews_df['Score'] >= 3, 'sentiment'] = 1
reviews_df.loc[reviews_df['Score'] < 3, 'sentiment'] = 0

In [7]:
# Q1.2. 

1.2. Clean the `Text` field by performing the following:
        1.2.1 make text lowercase
        1.2.2 remove text in square brackets,
        1.2.3 remove links,
        1.2.4 remove punctuation and 
        1.2.5 remove words containing numbers.

In [15]:
#convert the text in the Text column to lowercase

reviews_df['Text'] = reviews_df['Text'].str.lower()


In [16]:
#remove text in square brackets from the 'Text' column

reviews_df['Text'] = reviews_df['Text'].str.replace(r'[][]+', '', regex=True)

In [17]:
# remove links from the 'Text' column

reviews_df['Text'] = reviews_df['Text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')

In [18]:
# remove punctuation from the 'Text' column

reviews_df['Text']=reviews_df['Text'].str.replace('[{}]'.format(string.punctuation), '')

In [19]:
# remove words containing numbers from the 'Text' column

reviews_df['Text'] = reviews_df['Text'].str.replace('\d+', '')



In [13]:
# Q1.3. 

Add a new column to the dataframe called `token_list` and store a list of the individual words in the `Text` field
    
  

In [20]:
#tokenize the text in Text columnn and store in a new column called token_list
def apwords(words):
    filtered_sentence = []
    words = word_tokenize(words)
    for w in words:
        filtered_sentence.append(w)
    return filtered_sentence
addwords = lambda x: apwords(x)


reviews_df['token_list'] = reviews_df['Text'].apply(addwords)

In [21]:
#review the updated table

reviews_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment,token_list
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,i have bought several of the vitality canned d...,1.0,"[i, have, bought, several, of, the, vitality, ..."
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled as jumbo salted peanut...,0.0,"[product, arrived, labeled, as, jumbo, salted,..."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this is a confection that has been around a fe...,1.0,"[this, is, a, confection, that, has, been, aro..."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,if you are looking for the secret ingredient i...,0.0,"[if, you, are, looking, for, the, secret, ingr..."
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffy at a great price there was a wide...,1.0,"[great, taffy, at, a, great, price, there, was..."


In [16]:
# Q1.4. 

Remove the stopwords from the `token_list` column and store the results in a column called `no_stop_tokens`

In [22]:


from nltk.corpus import stopwords
stop = stopwords.words('english')

In [23]:
#remove stopwords from token_list and store in a new column called 'no_stop_tokens'

reviews_df['no_stop_tokens']= reviews_df['token_list'].apply(lambda x: [item for item in x if item not in stop])


In [24]:
# Q1.5. 

In [ ]:
Lemmatize the tokens in the `no_stop_tokens` field and store the results in a column called `Lemmatized_tokens`

In [25]:
lemmatizer = nltk.stem.WordNetLemmatizer()

In [26]:
#lemmatize the words in no_stop_tokens and store in a new column called 'lemmatized_tokens'

reviews_df['Lemmatized_tokens'] = reviews_df['no_stop_tokens'].apply(lambda x : [WordNetLemmatizer().lemmatize(y) for y in x])

In [27]:
# Q1.6. 

Create a bag of words of all lemmatized words in the dataframe, then find the top 10000 most common words and store them in a list called `top_words_list`

In [28]:
# Create the bag of words from lemmatized_tokens
count = CountVectorizer()
bag_of_words = count.fit_transform(reviews_df['Lemmatized_tokens'].astype(str))

In [29]:

sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in count.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    

In [30]:
#store top 10000 most frequent words in top_words_list

top_words_list=words_freq[:10000]

In [70]:
top_words_list

[('br', 12065),
 ('like', 11784),
 ('coffee', 9471),
 ('taste', 9002),
 ('good', 8732),
 ('one', 8279),
 ('flavor', 8006),
 ('product', 7496),
 ('love', 7181),
 ('great', 7081),
 ('food', 6287),
 ('dog', 6101),
 ('would', 5843),
 ('get', 5298),
 ('tea', 4998),
 ('make', 4731),
 ('really', 4453),
 ('dont', 4208),
 ('time', 4201),
 ('much', 4103),
 ('treat', 3902),
 ('bag', 3718),
 ('also', 3710),
 ('little', 3658),
 ('use', 3648),
 ('price', 3488),
 ('buy', 3411),
 ('tried', 3405),
 ('amazon', 3386),
 ('best', 3345),
 ('cup', 3343),
 ('im', 3249),
 ('even', 3246),
 ('try', 3187),
 ('better', 3165),
 ('find', 3123),
 ('well', 3058),
 ('ive', 3014),
 ('chip', 2969),
 ('drink', 2882),
 ('first', 2770),
 ('eat', 2735),
 ('store', 2624),
 ('sugar', 2538),
 ('box', 2534),
 ('chocolate', 2533),
 ('day', 2486),
 ('year', 2483),
 ('brand', 2467),
 ('found', 2376),
 ('made', 2361),
 ('give', 2333),
 ('cat', 2310),
 ('mix', 2294),
 ('used', 2266),
 ('go', 2264),
 ('way', 2248),
 ('think', 2237),
 

In [ ]:
Create a dictionary called `word_dict` 

In [31]:
# Q1.7. 

In [32]:
#function to create dictionary

def create_word_dict(top_words_list):
    
    word_dict = {}
    word_dict["<PAD>"] = 0
    word_dict["<START>"] = 1
    word_dict["<UNK>"] = 2
    word_dict["UNUSED"] = 3
    
    for i, word in enumerate(top_words_list):
        word_dict[word] = i+4
    
    return word_dict


In [33]:
# create a dictionary from top_words_list and store in word_dict

word_dict = create_word_dict(top_words_list)

In [71]:
word_dict

{'<PAD>': 0,
 '<START>': 1,
 '<UNK>': 2,
 'UNUSED': 3,
 ('br', 12065): 4,
 ('like', 11784): 5,
 ('coffee', 9471): 6,
 ('taste', 9002): 7,
 ('good', 8732): 8,
 ('one', 8279): 9,
 ('flavor', 8006): 10,
 ('product', 7496): 11,
 ('love', 7181): 12,
 ('great', 7081): 13,
 ('food', 6287): 14,
 ('dog', 6101): 15,
 ('would', 5843): 16,
 ('get', 5298): 17,
 ('tea', 4998): 18,
 ('make', 4731): 19,
 ('really', 4453): 20,
 ('dont', 4208): 21,
 ('time', 4201): 22,
 ('much', 4103): 23,
 ('treat', 3902): 24,
 ('bag', 3718): 25,
 ('also', 3710): 26,
 ('little', 3658): 27,
 ('use', 3648): 28,
 ('price', 3488): 29,
 ('buy', 3411): 30,
 ('tried', 3405): 31,
 ('amazon', 3386): 32,
 ('best', 3345): 33,
 ('cup', 3343): 34,
 ('im', 3249): 35,
 ('even', 3246): 36,
 ('try', 3187): 37,
 ('better', 3165): 38,
 ('find', 3123): 39,
 ('well', 3058): 40,
 ('ive', 3014): 41,
 ('chip', 2969): 42,
 ('drink', 2882): 43,
 ('first', 2770): 44,
 ('eat', 2735): 45,
 ('store', 2624): 46,
 ('sugar', 2538): 47,
 ('box', 2534):

Add a new column to the dataframe called `word_ids` calculated using the following function:
    ``python
    def word_to_id(words):
        return [1] + [word_dict[w] if w in word_dict.keys() else word_dict['<UNK>'] for w in words]
    ```
    

In [34]:
# Q1.8. 

In [35]:


def word_to_id(words):
     return [1] + [word_dict[w] if w in word_dict.keys() else word_dict['<UNK>'] for w in words]


Extract the `word_ids` and `sentiment` fields into a new dataframe called `dataset`.

In [36]:
#Add word_ids to a new column

reviews_df['word_ids']= reviews_df['Lemmatized_tokens'].apply(word_to_id)

In [37]:
# Q1.9. 

In [38]:
#extract word_ids and sentiments from reviews_df to dataset

dataset = reviews_df[['word_ids', 'sentiment']].copy()


In [39]:
#view columns in dataset
dataset.head()

,word_ids,sentiment
0,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
1,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.0
2,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
3,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.0
4,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",1.0


In [40]:
# Q2. 

Build a Feed Forward Model to predict the sentiment of each review. Then, evaluate the performance of the model in terms of loss and accuracy

In [57]:
Y= dataset['sentiment']

In [60]:
#Train and test data
vectorizer = TfidfVectorizer(stop_words='english', max_features=500, ngram_range=(1,1))
X_train = vectorizer.fit_transform(reviews_df['Text'])[:80000].A 
Y_train = dataset.sentiment[:80000] 
X_test = vectorizer.fit_transform(reviews_df['Text'])[80000:].A
Y_test = dataset.sentiment[80000:]


In [61]:

max_words = 500
X_train = keras.utils.pad_sequences(X_train, maxlen=max_words)
X_test = keras.utils.pad_sequences(X_test, maxlen=max_words)

In [62]:
#Feed forward model

model = Sequential()

model.add(keras.layers.Embedding(10000, 50, input_length=max_words))

model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [65]:
#model structure
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 50)           500000    
                                                                 
 flatten_2 (Flatten)         (None, 25000)             0         
                                                                 
 dense_9 (Dense)             (None, 32)                800032    
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,300,065
Trainable params: 1,300,065
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = model.fit(X_train, Y_train,
                   validation_data=(X_test, Y_test),
                   epochs=10,
                   batch_size=128,
                   verbose=1)

Epoch 1/10
196/196 [==============================] - 6s 30ms/step - loss: 0.4253 - accuracy: 0.8520
Epoch 2/10
196/196 [==============================] - 5s 27ms/step - loss: 0.4225 - accuracy: 0.8520
Epoch 3/10
196/196 [==============================] - 6s 29ms/step - loss: 0.4212 - accuracy: 0.8520
Epoch 4/10
196/196 [==============================] - 6s 32ms/step - loss: 0.4215 - accuracy: 0.8520
Epoch 5/10
196/196 [==============================] - 6s 33ms/step - loss: 0.4205 - accuracy: 0.8520
Epoch 6/10
196/196 [==============================] - 7s 36ms/step - loss: 0.4207 - accuracy: 0.8520
Epoch 7/10
196/196 [==============================] - 6s 30ms/step - loss: 0.4210 - accuracy: 0.8520
Epoch 8/10
196/196 [==============================] - 6s 30ms/step - loss: 0.4222 - accuracy: 0.8520
Epoch 9/10
196/196 [==============================] - 6s 30ms/step - loss: 0.4207 - accuracy: 0.8520
Epoch 10/10
196/196 [==============================] - 6s 32ms/step - loss: 0.4204 - accura

In [ ]:
# Q3. 

In [ ]:
Build a Convolutional Model to predict the sentiment of each review. Then, evaluate the performance of the model in terms of loss and accuracy


In [66]:
#Convolutional model

model = Sequential()
model.add(keras.layers.Embedding(10000, 50, input_length=max_words))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [67]:
#model structure

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 50)           500000    
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 32)           4832      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 32)          0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 8000)              0         
                                                                 
 dense_11 (Dense)            (None, 32)                256032    
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                      

In [69]:
history = model.fit(X_train, Y_train,
                   validation_data=(X_test, Y_test),
                   epochs=10,
                   batch_size=128,
                   verbose=1)

Epoch 1/10
196/196 [==============================] - 13s 65ms/step - loss: 0.4204 - accuracy: 0.8522
Epoch 2/10
196/196 [==============================] - 14s 71ms/step - loss: 0.4193 - accuracy: 0.8521
Epoch 3/10
196/196 [==============================] - 13s 68ms/step - loss: 0.4202 - accuracy: 0.8522
Epoch 4/10
196/196 [==============================] - 14s 70ms/step - loss: 0.4200 - accuracy: 0.8521
Epoch 5/10
196/196 [==============================] - 13s 69ms/step - loss: 0.4198 - accuracy: 0.8521
Epoch 6/10
196/196 [==============================] - 14s 69ms/step - loss: 0.4197 - accuracy: 0.8521
Epoch 7/10
196/196 [==============================] - 14s 70ms/step - loss: 0.4199 - accuracy: 0.8522
Epoch 8/10
196/196 [==============================] - 14s 70ms/step - loss: 0.4195 - accuracy: 0.8522
Epoch 9/10
196/196 [==============================] - 14s 72ms/step - loss: 0.4195 - accuracy: 0.8522
Epoch 10/10
196/196 [==============================] - 14s 71ms/step - loss: 0.419

In [ ]:
Which model performs better and why?

##### Q4. 

The convolutional model produces a higher accuracy of 0.8522 and a lower loss of 0.4204 while feed forward model has an accuracy of 0.8518 and a loss of 0.4256.

The convolutional method has an higher accuracy than feed-forward  because it has capabilities such as parameter sharing and dimensionality reduction.

The number of parameters in a convolutional model is lowered as a result of parameter sharing, and so the calculations are also reduced.

<div class="alert alert-block alert-info">

# Part 2: Reflection
    
</div>

1. The initial preparation of data was quite easy as I had performed this activity in previous labs. However training the models proved to be quite difficult as I was not able to import the keras and tensor flow modules. I eventually was able to download the modules after reading several suggestions online. Afterwards the models were easy to train and test.

2. It is important to learn how to build models because models help in identifying significant features from datsets without human supervision.
